In [1]:

import numpy as np

# mne imports
import mne
from mne import io
from mne.datasets import sample

# EEGNet-specific imports
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.viz import plot_confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt

from dataloader import get_loader
from sklearn.model_selection import train_test_split

# while the default tensorflow ordering is 'channels_last' we set it here
# to be explicit in case if the user has changed the default ordering
K.set_image_data_format('channels_last')


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()




In [26]:
root = '/Volumes/Datasets/inner_speech/derivatives/'
# root =  'dataset/derivatives/' # -sil
creater = get_loader(root, n_sess= 3)

# extract raw data. scale by 1000 due to scaling sensitivity in deep learning
X, y = creater.load_multiple_subjects([1, 2, 4, 5, 6, 7]) # format is in (trials, channels, samples)
y = y[:, 1]
#X = X*1000

"""
import numpy as np
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt


def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y


# Setting standard filter requirements.
order = 10
fs = 1024       
cutoff = 15  


# Filtering and plotting
X = butter_lowpass_filter(X, cutoff, fs, order)

X = X*1000
"""

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_validate, X_test, Y_validate, Y_test = train_test_split(X_test, Y_test, test_size=0.50, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
X_validate = scaler.transform(X_validate.reshape(-1, X_validate.shape[-1])).reshape(X_validate.shape)


kernels, chans, samples = 1, 5, 128

N, _, _ = X.shape



In [29]:
import pandas as pd
import numpy as np
X = X*1000
stat_data = []

for i in range(X_train.shape[0]):
    data = X_train[i, :, :]*1000    
    df_describe = pd.DataFrame(data.T)
    stat_data.append(df_describe.describe()) 
    
X_train = np.stack(stat_data)

ValueError: cannot reshape array of size 1056768 into shape (5,8)

In [32]:
stat_data = []

for i in range(X_validate.shape[0]):
    data = X_validate[i, :, :]*1000    
    df_describe = pd.DataFrame(data.T)
    stat_data.append(df_describe.describe()) 
    
X_validate = np.stack(stat_data)

In [33]:
stat_data = []

for i in range(X_test.shape[0]):
    data = X_test[i, :, :]*1000    
    df_describe = pd.DataFrame(data.T)
    stat_data.append(df_describe.describe()) 
    
X_test = np.stack(stat_data)

In [37]:
samples = 128
chans = 8
# convert labels to one-hot encodings.
Y_train      = np_utils.to_categorical(Y_train-1)
Y_validate   = np_utils.to_categorical(Y_validate-1)
Y_test       = np_utils.to_categorical(Y_test-1)

# convert data to NHWC (trials, channels, samples, kernels) format. Data 
# contains 60 channels and 151 time-points. Set the number of kernels to 1.
X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
X_validate   = X_validate.reshape(X_validate.shape[0], chans, samples, kernels)
X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)
   
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (1032, 8, 128, 1)
1032 train samples
172 test samples


In [45]:
# configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
# model configurations may do better, but this is a good starting point)
model = EEGNet(nb_classes = 4, Chans = 8, Samples = 128, 
               dropoutRate = 0.5, kernLength = 128, F1 = 2, D = 2, F2 = 4, 
               dropoutType = 'Dropout')

In [46]:
import tensorflow_addons as tfa

# define the loss function 
loss_function = tfa.losses.SigmoidFocalCrossEntropy()

# compile the model and set the optimizers
model.compile(loss=loss_function, optimizer='adam', 
              metrics = ['accuracy'])

# count number of parameters in the model
numParams    = model.count_params()    

# set a valid path for your system to record model checkpoints
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                               save_best_only=True)

print(numParams)

476


In [47]:
###############################################################################
# if the classification task was imbalanced (significantly more trials in one
# class versus the others) you can assign a weight to each class during 
# optimization to balance it out. This data is approximately balanced so we 
# don't need to do this, but is shown here for illustration/completeness. 
###############################################################################

# the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
# the weights all to be 1
class_weights = {0:1, 1:1, 2:1, 3:1}

################################################################################
# fit the model. Due to very small sample sizes this can get
# pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
# Riemannian geometry classification (below)
################################################################################
fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 300, 
                        verbose = 2, validation_data=(X_validate, Y_validate),
                        callbacks=[checkpointer])

Epoch 1/300


ValueError: in user code:

    /Users/karanthakkar/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /Users/karanthakkar/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/karanthakkar/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/karanthakkar/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/karanthakkar/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/karanthakkar/opt/anaconda3/lib/python3

In [56]:
output_predicted = model.predict(X_validate[:,:,2])
accuracy = sum(output_predicted == Y_validate)/len(output_predicted)
accuracy

0.20930232558139536

In [57]:
output_predicted = model.predict(X_test[:,:,2])
accuracy = sum(output_predicted == Y_test)/len(output_predicted)
accuracy

0.18604651162790697